In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet121
from keras.models import Model
import keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet121
from keras.models import Model
from sklearn.metrics import roc_auc_score
import cv2
import keras.backend as K
import os
import random
from sklearn.utils import shuffle
from keras.applications.mobilenetv2 import MobileNetV2
from keras.callbacks import Callback,ModelCheckpoint
import xgboost as xgb
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
%matplotlib inline

Using TensorFlow backend.


In [2]:
Y_train=np.load('/home/dev/t1/ClassificationModel_SharedData/Padded_15ClassData_DMH+NIH+Normal/Y_train.npy', mmap_mode='r+')
Y_test=np.load('/home/dev/t1/ClassificationModel_SharedData/Padded_15ClassData_DMH+NIH+Normal/Y_test.npy',mmap_mode='r+')

In [3]:
feat_train=np.load('./feat_train.npy', mmap_mode='r+')
feat_test=np.load('./feat_test.npy', mmap_mode='r+')

In [ ]:
dtrain = xgb.DMatrix(feat_train, label=Y_train)
dtest = xgb.DMatrix(feat_test, label=Y_test)


In [ ]:
param = { 'max_depth': 100,'eta': 0.3, 'objective': 'multi:softmax', 'num_class': 15}
num_round = 30  # the number of training iterations

In [ ]:
watchlist=[(dtrain, 'train'), (dtest, 'valid')]
xgboost_model=xgb.train(param, dtrain, num_round, evals = watchlist,
              verbose_eval=1, maximize=True)

In [6]:
#xgboost_model = xgb.XGBClassifier()
#xgboost_model.fit(X_train, Y_train)

ValueError: bad input shape (13118, 15)

In [1]:
xgboost_model.dump_model('Xgboost_trained_finally.txt')

NameError: name 'xgboost_model' is not defined

In [13]:
preds = xgboost_model.predict(dtest)

In [16]:
test_preds = preds

In [17]:
train_preds = xgboost_model.predict(dtrain)

In [18]:
train_preds

array([[0.8422    , 0.07821329, 0.00612206, ..., 0.00612206, 0.00612206,
        0.00612206],
       [0.857034  , 0.06389423, 0.00608244, ..., 0.00608244, 0.00608244,
        0.00608244],
       [0.84429806, 0.07591686, 0.00613731, ..., 0.00613731, 0.00613731,
        0.00613731],
       ...,
       [0.8209729 , 0.10144626, 0.00596776, ..., 0.00596776, 0.00596776,
        0.00596776],
       [0.8414864 , 0.0789943 , 0.00611687, ..., 0.00611687, 0.00611687,
        0.00611687],
       [0.8396227 , 0.08103411, 0.00610332, ..., 0.00610332, 0.00610332,
        0.00610332]], dtype=float32)

In [23]:
accuracy = accuracy_score(Y_train, train_preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [24]:
Y_train

memmap([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
test_preds

array([[0.8410607 , 0.07946017, 0.00611378, ..., 0.00611378, 0.00611378,
        0.00611378],
       [0.8547244 , 0.06450531, 0.0062131 , ..., 0.0062131 , 0.0062131 ,
        0.0062131 ],
       [0.8524958 , 0.06885119, 0.00605023, ..., 0.00605023, 0.00605023,
        0.00605023],
       ...,
       [0.8359261 , 0.08508001, 0.00607645, ..., 0.00607645, 0.00607645,
        0.00607645],
       [0.8335703 , 0.08952279, 0.00591592, ..., 0.00591592, 0.00591592,
        0.00591592],
       [0.82279897, 0.0994477 , 0.00598103, ..., 0.00598103, 0.00598103,
        0.00598103]], dtype=float32)

In [26]:
global_cls = []
pad_f1s = {}
avg_f1s = []
for d in range(15):
    try:
        score = f1_score(Y_test[:,d], np.around(test_preds[:,d]))
        pad_f1s[global_cls[d]] = score
        avg_f1s.append(score*np.sum(Y_test[:,d]))
    except:
        print(global_cls[d])
pad_f1s